## 필요한 라이브러리 로드

In [3]:
import pandas as pd
import numpy as np

In [4]:
pd.options.display.float_format = '{:.5f}'.format

금액과 관련된 변수를 지수형태로 나타나지 않게 위한 작업을 시행한다.

## csv파일 데이터 로드 및 변수 선택

In [5]:
df_gg = pd.read_csv('/content/물품입찰공고.csv') #물품입찰공고 # 파일 경로 작성 필요
df_cf = pd.read_csv('/content/물품입찰분류별진행내역.csv') #물품입찰분류별진행내역 # 파일 경로 작성 필요

In [6]:
df_gg_features = df_gg[['등록유형','조달구분', '입찰공고번호', '품명내용', '긴급공고여부', '추정가격', '배정예산', 
                        '공동도급협정서접수방식', '입찰방식','낙찰자결정방법', '낙찰하한율']]

df_cf_features = df_cf[['입찰공고번호', '합계수량', '낙찰자결정여부', '낙찰업체투찰률', '낙찰업체투찰금액', '낙찰자결정적용법규', '기초금액', '참가수']]

모델링 전 전처리 과정에서 실제로 null값을 가진 값을 drop, data의 value 대체 등 조건을 걸어 전처리를 진행한 변수들만 선택하였다.

In [7]:
df_2 = pd.merge(df_gg_features, df_cf_features, how='inner', on='입찰공고번호')

In [8]:
df = df_2.copy()

merge를 통하여 두 가지 csv파일에서 변수를 선택한 후 입찰공고번호를 기준으로 inner join하여 하나의 데이터프레임으로 합병한다.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108349 entries, 0 to 108348
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   등록유형         108349 non-null  object 
 1   조달구분         108349 non-null  object 
 2   입찰공고번호       108349 non-null  int64  
 3   품명내용         108348 non-null  object 
 4   긴급공고여부       91583 non-null   object 
 5   추정가격         105957 non-null  float64
 6   배정예산         104871 non-null  float64
 7   공동도급협정서접수방식  108147 non-null  object 
 8   입찰방식         108349 non-null  object 
 9   낙찰자결정방법      106851 non-null  object 
 10  낙찰하한율        64600 non-null   float64
 11  합계수량         108349 non-null  float64
 12  낙찰자결정여부      108349 non-null  object 
 13  낙찰업체투찰률      85136 non-null   float64
 14  낙찰업체투찰금액     85136 non-null   float64
 15  낙찰자결정적용법규    101608 non-null  object 
 16  기초금액         67274 non-null   float64
 17  참가수          85136 non-null   float64
dtypes: float64(8), int64(1),

## data 전처리

In [10]:
df = df[df['등록유형'] == '나라장터(G2B)']
df = df.drop(['등록유형'], axis=1)

등록유형을 나라장터로 한정하고, 등록유형을 drop한다.

In [11]:
df = df.dropna(subset=['품명내용'])
df = df.dropna(subset=['긴급공고여부'])
df = df.dropna(subset=['추정가격'])
df = df.dropna(subset=['배정예산'])

품명내용, 긴급공고여부, 추정가격, 배정예산이 na인 값은 대체가 불가능하므로 drop한다.

In [12]:
df = df[df['입찰방식'].str.contains('전자')]
df = df.drop(['입찰방식'], axis=1)

df.head(3)

,조달구분,입찰공고번호,품명내용,긴급공고여부,추정가격,배정예산,공동도급협정서접수방식,낙찰자결정방법,낙찰하한율,합계수량,낙찰자결정여부,낙찰업체투찰률,낙찰업체투찰금액,낙찰자결정적용법규,기초금액,참가수
0,중앙조달,20210121987,액체크로마토그래프[41115705],Y,150000000.00000,166564000.00000,일반(중앙),제안서적격자중 예가내 최저투찰자,NaN,1.00000,N,0.00000,0.00000,국가계약법,NaN,2.00000
1,자체조달,20210123012,전광판[55121903],N,50972727.00000,56070000.00000,일반(중앙),제한적최저가(낙찰하한율),88.00000,1.00000,Y,88.02300,49439300.00000,지방계약법,56070000.00000,19.00000
2,자체조달,20210123612,곡류가루[50221102],N,622934400.00000,685227840.00000,일반(중앙),적격심사(추정가격 고시금액이상 10억원 미만),80.49500,1.00000,Y,84.22800,579500000.00000,지방계약법,685227840.00000,4.00000


전자입찰과 관련된 입찰방식으로 추린다.

In [13]:
df['낙찰하한율'].isna().sum() #낙찰하한율 대체 전

23603

null값의 대체가 가능한 낙찰하한율은 대체한다.

In [14]:
df[(df['낙찰자결정방법'].isna()==True) & (df['낙찰하한율'].isna()==False)][['낙찰자결정방법', '낙찰하한율']]

,낙찰자결정방법,낙찰하한율
400,NaN,87.99500
1096,NaN,90.00000
3164,NaN,87.99500
3972,NaN,88.00000
4125,NaN,87.99500
...,...,...
106535,NaN,87.99500
106622,NaN,87.99500
107751,NaN,90.00000
108029,NaN,87.99500


낙찰자결정방법을 통해 낙찰하한율을 대체할 예정이다. 그러나 낙찰자결정방법이 결측치인 데이터를 모두 제거하면, 낙찰자결정방법은 결측이지만 낙찰하한율은 결측이 아닌 데이터까지 제거하게 되므로 낙찰하한율이 있는 경우에는 낙찰자결정방법을 drop하지 않는다.

In [15]:
df['낙찰자결정방법'] = df['낙찰자결정방법'].fillna('알수없음')

In [16]:
df.loc[df['낙찰자결정방법'].str.contains('10억이상'), '낙찰하한율'] = 80.495
df.loc[df['낙찰자결정방법'].str.contains('10억원이상'), '낙찰하한율'] = 80.495

df.loc[df['낙찰자결정방법'].str.contains('10억원미만'), '낙찰하한율'] = 80.495
df.loc[df['낙찰자결정방법'].str.contains('10억미만'), '낙찰하한율'] = 80.495

df.loc[df['낙찰자결정방법'].str.contains('고시금액미만'), '낙찰하한율'] = 84.245
df.loc[df['낙찰자결정방법'].str.contains('고시금액 미만'), '낙찰하한율'] = 84.245

df.loc[df['낙찰자결정방법'].str.contains('간행물'), '낙찰하한율'] = 89.995

낙찰하한율을 대체할 수 있는 정보가 한정적이여서 많이 대체하지는 못했지만, 정보가 업데이트 된다면 모델을 업데이트 할 필요가 있어보임.

In [17]:
df['낙찰하한율'].isna().sum() #낙찰하한율 대체 후

23063

낙찰하한율의 null값의 개수가 감소하였음.

In [18]:
df = df.dropna(subset=['낙찰하한율'])

낙찰하한율의 나머지 null값은 처리가 불가능하므로 drop한다.

In [19]:
df = df.dropna(subset=['기초금액'])
df = df.dropna(subset=['추정가격'])
df = df.dropna(subset=['낙찰업체투찰금액'])

금액과 관련된 변수는 대체 시 data상의 오류가 발생할 가능성이 있으므로 null값을 drop한다.

금액과 관련된 변수는 null값이 없어야 하며 대체도 불가능하다.

In [20]:
df = df.dropna(subset=['합계수량'])
df = df.dropna(subset=['낙찰자결정적용법규'])
df = df.dropna(subset=['공동도급협정서접수방식'])
df = df.dropna(subset=['참가수'])
df = df[df['조달구분']=='자체조달']

df.head(3)

,조달구분,입찰공고번호,품명내용,긴급공고여부,추정가격,배정예산,공동도급협정서접수방식,낙찰자결정방법,낙찰하한율,합계수량,낙찰자결정여부,낙찰업체투찰률,낙찰업체투찰금액,낙찰자결정적용법규,기초금액,참가수
1,자체조달,20210123012,전광판[55121903],N,50972727.00000,56070000.00000,일반(중앙),제한적최저가(낙찰하한율),88.00000,1.00000,Y,88.02300,49439300.00000,지방계약법,56070000.00000,19.00000
2,자체조달,20210123612,곡류가루[50221102],N,622934400.00000,685227840.00000,일반(중앙),적격심사(추정가격 고시금액이상 10억원 미만),80.49500,1.00000,Y,84.22800,579500000.00000,지방계약법,685227840.00000,4.00000
3,자체조달,20210123753,생고기또는생새고기류[50111598],N,84481818.00000,92930000.00000,일반(중앙),적격심사(추정가격 고시금액미만),84.24500,1.00000,Y,84.42200,78410000.00000,지방계약법,92930000.00000,5.00000


합계수량과 낙찰자 결정 적용 법규, 공동도급 협정서 접수 방식, 참가수를 다른 값으로 대체한다면 data상의 오류가 발생할 가능성이 있다. 따라서 drop하고, EDA 당시 중앙조달의 data 수가 매우 적어 가격 예측에 도움이 되지 못할 것 같아 자체조달로 한정하였으므로 자체조달로 한정한다.

In [21]:
df = df[df['낙찰업체투찰금액'] != 0]

낙찰업체투찰금액이 0원인 값은 Null로 간주하여 0원을 제거한다.

In [22]:
Q1 = np.percentile(df['낙찰업체투찰금액'].values, 25)
Q3 = np.percentile(df['낙찰업체투찰금액'].values, 75)

IQR = Q3-Q1

df = df[(df['낙찰업체투찰금액'] >= (Q1 - (1.5*IQR))) & (df['낙찰업체투찰금액'] <= (Q3 + (1.5*IQR)))]

낙찰업체투찰금액의 편차가 매우 크므로, 이상치를 제거한다.

In [23]:
df = df[df['낙찰하한율'] >= 79]

df.head(3)

,조달구분,입찰공고번호,품명내용,긴급공고여부,추정가격,배정예산,공동도급협정서접수방식,낙찰자결정방법,낙찰하한율,합계수량,낙찰자결정여부,낙찰업체투찰률,낙찰업체투찰금액,낙찰자결정적용법규,기초금액,참가수
1,자체조달,20210123012,전광판[55121903],N,50972727.00000,56070000.00000,일반(중앙),제한적최저가(낙찰하한율),88.00000,1.00000,Y,88.02300,49439300.00000,지방계약법,56070000.00000,19.00000
3,자체조달,20210123753,생고기또는생새고기류[50111598],N,84481818.00000,92930000.00000,일반(중앙),적격심사(추정가격 고시금액미만),84.24500,1.00000,Y,84.42200,78410000.00000,지방계약법,92930000.00000,5.00000
7,자체조달,20210124885,상온보관우유또는버터제품[50131702],N,80410000.00000,88451000.00000,일반(중앙),제한적최저가(낙찰하한율),84.24500,205700.00000,Y,84.25200,363.28000,지방계약법,430.00000,224.00000


낙찰하한율이 79% 이하인 것은 이상치로 판단하고 79 이상인 것들로만 간추린다.

In [24]:
df.loc[df['낙찰자결정방법'].str.contains('적격심사'), '낙찰자결정방법'] = '적격심사낙찰제'
df.loc[df['낙찰자결정방법'].str.contains('최저가'), '낙찰자결정방법'] = '최저가낙찰제'
df.loc[df['낙찰자결정방법'].str.contains('계약이행능력심사'), '낙찰자결정방법'] = '계약이행능력심사'
df.loc[df['낙찰자결정방법'].str.contains('희망수량경쟁'), '낙찰자결정방법'] = '희망수량경쟁'

df.낙찰자결정방법.value_counts()

최저가낙찰제      31023
적격심사낙찰제      6990
계약이행능력심사     1527
알수없음           34
희망수량경쟁         19
Name: 낙찰자결정방법, dtype: int64

낙찰자 결정방법의 명칭을 통일한다.

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39593 entries, 1 to 108331
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   조달구분         39593 non-null  object 
 1   입찰공고번호       39593 non-null  int64  
 2   품명내용         39593 non-null  object 
 3   긴급공고여부       39593 non-null  object 
 4   추정가격         39593 non-null  float64
 5   배정예산         39593 non-null  float64
 6   공동도급협정서접수방식  39593 non-null  object 
 7   낙찰자결정방법      39593 non-null  object 
 8   낙찰하한율        39593 non-null  float64
 9   합계수량         39593 non-null  float64
 10  낙찰자결정여부      39593 non-null  object 
 11  낙찰업체투찰률      39593 non-null  float64
 12  낙찰업체투찰금액     39593 non-null  float64
 13  낙찰자결정적용법규    39593 non-null  object 
 14  기초금액         39593 non-null  float64
 15  참가수          39593 non-null  float64
dtypes: float64(8), int64(1), object(7)
memory usage: 5.1+ MB


data의 전처리는 진행되었으므로, 필요한 변수를 제외하고 전부 삭제한다.

In [26]:
df = df[['합계수량', '기초금액', '추정가격', '낙찰하한율', '낙찰업체투찰금액']]

df.head(3)

,합계수량,기초금액,추정가격,낙찰하한율,낙찰업체투찰금액
1,1.00000,56070000.00000,50972727.00000,88.00000,49439300.00000
3,1.00000,92930000.00000,84481818.00000,84.24500,78410000.00000
7,205700.00000,430.00000,80410000.00000,84.24500,363.28000


전처리용 column을 제외하고, 실제 모델링에 필요한 변수들만 간추린다.

In [27]:
df['개당기초금액'] = df['기초금액'] / df['합계수량']
df['개당추정가격'] = df['추정가격'] / df['합계수량']
df['개당낙찰업체투찰금액'] = df['낙찰업체투찰금액'] / df['합계수량']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [28]:
df.head(3)

,합계수량,기초금액,추정가격,낙찰하한율,낙찰업체투찰금액,개당기초금액,개당추정가격,개당낙찰업체투찰금액
1,1.00000,56070000.00000,50972727.00000,88.00000,49439300.00000,56070000.00000,50972727.00000,49439300.00000
3,1.00000,92930000.00000,84481818.00000,84.24500,78410000.00000,92930000.00000,84481818.00000,78410000.00000
7,205700.00000,430.00000,80410000.00000,84.24500,363.28000,0.00209,390.90909,0.00177


In [29]:
df.isnull().sum()

합계수량          0
기초금액          0
추정가격          0
낙찰하한율         0
낙찰업체투찰금액      0
개당기초금액        0
개당추정가격        0
개당낙찰업체투찰금액    0
dtype: int64

각 data마다 합계수량의 차이가 있으므로, 금액과 관련된 변수를 합계수량으로 나누어 개당 금액을 구한다.

# Mini Project(경고창으로 입찰 성공률 높이기)



나라장터에서 다양한 공고의 개찰 결과를 확인했을 때, 낙찰하한선 미달, 예가 초과와 같은 이유로 심사에서 탈락한 경우를 확인했다. 특히, 나라장터를 이용한 공공조달을 처음 이용하는 개인이나 기업의 입장에서 투찰금액을 제시하는 방법을 몰라 심사에서 탈락하는 경우가 많을 것이라 예상된다. 따라서, 나라장터를 처음 이용하거나, 입찰 경험이 없는 사람들을 위해 제시한 투찰금액이 낙찰하한선 미달이나 예가 초과의 가능성이 있을 경우, 경고창을 띄어 입찰 가능성을 높일 수 있다는 기대를 하였다.

In [30]:
df_check = df.copy() # Mini project에 사용할 데이터프레임 생성.

In [31]:
df_check.reset_index(drop=True, inplace=True) # index를 내림차순으로 새로 설정하였다.

In [35]:
df_check.drop(['합계수량'], axis=1, inplace=True)
# 사용할 변수를 제외한 나머지 변수를 drop한다.

기초금액을 이용한 예정가격 산정 순서
1. 기초금액을 기준으로 98%~102% 사이의 복수예가 15개 생성
2. 복수예가 15개 중 추첨을 통해 4개 선택
3. 선택된 4개의 숫자를 산술평균하여 예정가격 산정

위 과정을 코드로 작성하여 기초금액을 통해 예정가격을 예측하였다.

In [36]:
import random

lists=[]
for j in range(0, len(df_check)):  
    for i in range(1, 16):
        number = [random.uniform(df_check['개당기초금액'][j]*0.98, df_check['개당기초금액'][j]*1.02)] # +-2% 범위 내에서 15개의 난수 생성
        select = np.random.choice(number, 4) # 15개의 난수 중 임의로 4개 선택
        average = np.mean(select) # 선택한 4개 숫자의 산술평균: 예정가격
    lists.append(average)

In [37]:
df_check['예정가격_pred'] = lists # 예측된 예정가격을 '예정가격_pred' column에 저장하였다.

In [38]:
df_check['낙찰하한선_pred'] = df_check['예정가격_pred'] * df_check['낙찰하한율'] * 0.01

In [39]:
df_check['낙찰하한선_기초'] = df_check['개당기초금액'] * df_check['낙찰하한율'] * 0.01

In [40]:
df_check['낙찰하한선_추정'] = df_check['개당추정가격'] * df_check['낙찰하한율'] * 0.01

예정가격은 입찰 공고 단계에서 확인할 수 없으므로 우리가 사용할 수 없는 볍수다. 따라서 기초금액을 통해 예정가격을 직접 예측해보고 이를 통해 낙찰하한선 미달과 예가초과의 범위를 시험해볼 것이다.

크게 3가지 Case로 구분해서 test할 것이다.
1.   직접 구한 예정가격을 이용한 test
2.   기초금액을 이용한 test
3.   추정가격을 이용한 test


In [41]:
df_check.head()

,기초금액,추정가격,낙찰하한율,낙찰업체투찰금액,개당기초금액,개당추정가격,개당낙찰업체투찰금액,예정가격_pred,낙찰하한선_pred,낙찰하한선_기초,낙찰하한선_추정
0,56070000.00000,50972727.00000,88.00000,49439300.00000,56070000.00000,50972727.00000,49439300.00000,57188391.59350,50325784.60228,49341600.00000,44855999.76000
1,92930000.00000,84481818.00000,84.24500,78410000.00000,92930000.00000,84481818.00000,78410000.00000,92684406.09432,78081977.91416,78288878.50000,71171707.57410
2,430.00000,80410000.00000,84.24500,363.28000,0.00209,390.90909,0.00177,0.00211,0.00178,0.00176,329.32136
3,430.00000,430.00000,88.00000,372.44000,0.00283,0.00283,0.00245,0.00286,0.00252,0.00249,0.00249
4,430.00000,430.00000,88.00000,387.12000,0.00208,0.00208,0.00187,0.00206,0.00181,0.00183,0.00183


1. 직접 구한 예정가격을 이용한 test

In [42]:
count=0
for i in range(0, len(df)):
    if df_check['낙찰하한선_pred'][i] > df_check['개당낙찰업체투찰금액'][i]: # 낙찰하한선 미달
        count+=1
print(count)
        #print('경고: {}번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.'.format(i))

13181


In [43]:
count=0
for i in range(0, len(df)):
    if df_check['예정가격_pred'][i] < df_check['개당낙찰업체투찰금액'][i]: # 예가초과
        count+=1
print(count)
        #print('경고: {}번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.'.format(i))

528


예정가격은 기초금액의 +-2%내에서 생성된 난수를 통해 산정되기 때문에 새로운 예정가격이 산정될 때 마다 test 값이 달라진다. 다만, 어느정도 범위가 한정되어 있어 오차값이 크지는 않다.

2. 기초금액을 이용한 test

In [44]:
count=0
for i in range(0, len(df)):
    if df_check['낙찰하한선_기초'][i] > df_check['개당낙찰업체투찰금액'][i]: # 낙찰하한선 미달
        count+=1
print(count)
        #print('경고: {}번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.'.format(i))

12357


In [45]:
count=0
for i in range(0, len(df)):
    if df_check['개당기초금액'][i] < df_check['개당낙찰업체투찰금액'][i]: # 예가초과
        count+=1
print(count)
        #print('경고: {}번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.'.format(i))

17


3. 추정가격을 이용한 test

In [46]:
count=0
for i in range(0, len(df)):
    if df_check['낙찰하한선_추정'][i] > df_check['개당낙찰업체투찰금액'][i]: # 낙찰하한선 미달
        count+=1
print(count)
        #print('경고: {}번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.'.format(i))

5553


In [47]:
count=0
for i in range(0, len(df)):
    if df_check['개당추정가격'][i] < df_check['개당낙찰업체투찰금액'][i]: # 예가초과
        count+=1
print(count)
        #print('경고: {}번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.'.format(i))

7586


Result

추정가격은 기초금액이나 예정가격에 비해 과하게 측정되는 경우가 많다. 따라서 낙찰하한선 미달의 경우에는 좋은 결과를 보여주지만 예가초과의 경우에는 좋지 않는 결과를 보여준다. 따라서, 추정가격을 사용하는 것은 옳지 않다고 본다.

기초금액과 직접 구한 예정가격을 비교해보면, 기초금액의 결과가 조금 더 좋은 것을 확인할 수 있다. 하지만, 기초금액을 통해 낙찰하한선 미달과 예가 초과를 예측하는 것은 사실상 쉽지 않다. 그리고 우리가 사용한 데이터는 대략 4만개 정도 이므로 매우 적은 수치다. 더 많은 데이터를 사용한다면 예정가격을 사용했을 때 더 좋은 결과를 보일 수도 있을 것이라 판단되므로 Mini project에서는 예정가격을 사용하기로 결론 짓는다.

# Final Code

In [49]:
df_check = df.copy() # Mini project에 사용할 데이터프레임 생성.

df_check.reset_index(drop=True, inplace=True) # index를 내림차순으로 새로 설정하였다.

df_check.drop(['합계수량'], axis=1, inplace=True)
# 사용할 변수를 제외한 나머지 변수를 drop한다.

In [50]:
import random

lists=[]
for j in range(0, len(df_check)):  
    for i in range(1, 16):
        number = [random.uniform(df_check['개당기초금액'][j]*0.98, df_check['개당기초금액'][j]*1.02)] # +-2% 범위 내에서 15개의 난수 생성
        select = np.random.choice(number, 4) # 15개의 난수 중 임의로 4개 선택
        average = np.mean(select) # 선택한 4개 숫자의 산술평균: 예정가격
    lists.append(average)

In [51]:
df_check['예정가격_pred'] = lists # 예측된 예정가격을 '예정가격_pred' column에 저장하였다.
df_check['낙찰하한선_pred'] = df_check['예정가격_pred'] * df_check['낙찰하한율'] * 0.01

In [52]:
for i in range(0, len(df)):
    if df_check['낙찰하한선_pred'][i] > df_check['개당낙찰업체투찰금액'][i]: # 낙찰하한선 미달
        print('경고: {}번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.'.format(i))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
경고: 24124번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24125번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24129번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24134번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24136번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24138번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24142번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24143번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24144번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24145번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24149번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24150번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24156번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24158번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24159번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24160번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24164번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24168번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24170번째 투찰금액은 낙찰하한선 미달이므로 이후 심사에서 탈락할 수 있습니다.
경고: 24174번째 투찰

In [53]:
for i in range(0, len(df)):
    if df_check['예정가격_pred'][i] < df_check['개당낙찰업체투찰금액'][i]: # 예가초과
        print('경고: {}번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.'.format(i))

경고: 41번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 282번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 447번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 536번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 554번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 677번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 765번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 814번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 1122번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 1187번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 1201번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 1202번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 1215번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 1304번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 1329번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 2072번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 2209번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 2293번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 2327번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 2372번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 2616번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니다.
경고: 2625번째 투찰금액은 예가 초과이므로 이후 심사에서 탈락할 수 있습니